In [1]:
from matplotlib.colors import ListedColormap # for grgphing decision boundaries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split

In [2]:
data_folder = 'data'
X_train = pd.read_csv(f'./{data_folder}/X_train.csv')
y_train = pd.read_csv(f'./{data_folder}/y_train.csv')
X_test = pd.read_csv(f'./{data_folder}/X_test.csv')
y_test = pd.read_csv(f'./{data_folder}/y_test.csv')

In [3]:
class Network:
    def __init__(self, sample_input, num_layers, neurons_per_layer):
        self.layers = self.create_layers(sample_input, num_layers, neurons_per_layer)
        self.weights = self.create_weights()
        self.biases = [[0]*self.layers[i] for i in range(1, len(self.layers))]
        
    @classmethod
    def create_layers(self, sample_input, num_layers, neurons_per_layer):
        """ function to create hidden layers, input/output layers """
        input_layer = [0 for i in sample_input]
        output_layer = [0 for i in range(3)]
        layers = [0] * (num_layers+1)
        layers[0] = np.reshape(input_layer,(-1,1))
        # for num of layers
        for i in range(num_layers):
            # create a empty list for neurons
            current_layer = [[0] for j in  range(neurons_per_layer[i])]
            layers[i+1] = np.reshape(current_layer, (-1,1))
        layers.append(np.reshape(output_layer, (-1,1)))
        return layers

    def create_weights(self):
        weights = []
        layers = self.layers
        for i in range(len(layers)-1):
            # get the length of the current layer
            current_layer = len(layers[i])
            # get the length of the next layer
            next_layer = len(layers[i+1])
            # create a matrix and append to list using the lengths of the layers 
            weights.append(np.random.rand(next_layer, current_layer))
        return weights 
        
        
    def sigmoid(self, activation):
        return (1/(1+np.exp(-(activation))))
    
    def feed_forward(self, X):
        # select the layers
        layers = self.layers
        # set the input as the input layer
        layers[0] = np.reshape(X, (-1,1))
        # feed the input forward
        for i in range(len(layers)-1):
#             print(f'weights: {self.weights[i]}')
            activations = self.sigmoid(np.dot(self.weights[i], layers[i]) + self.biases[i])
            layers[i+1] = activations.copy()
        return (layers[-1], max(layers[-1]))
    
    
    def back_propagation(self, output, y, l_rate):
        layers = self.layers
        weights = self.weights
        biases = self.biases
        cost_function = 2 * (layers[-1]-y)
        # propogate the error from the output to previous layer
        weights[-1] = -l_rate*np.dot(cost_function, layers[-2].T)
        biases[-1] = -l_rate*cost_function 
        
        # propogate error from second hidden layer to first hidden layer
        num = np.dot(weights[-1].T, cost_function) * (self.sigmoid(layers[-2])*(self.sigmoid(layers[-2])))
        weights[-2] = -l_rate*np.dot(num, layers[-4].T)
        biases[-2] = -l_rate*num 
        
        # propogate error from first hidden layer to output layer
        other_num = np.dot(weights[-2].T, cost_function) * (self.sigmoid(layers[-3])*(self.sigmoid(layers[-3])))
        weights[-2] = -l_rate*np.dot(num, layers[0].T)
        biases[0] = -l_rate*other_num
        
        return 0
    
    def train(self, X, y):
        y_dummies = pd.get_dummies(y)
        data = pd.concat([X, y_dummies], axis=1)
        for index, row in data.iterrows():
            y = np.reshape(row.tolist()[4:], (-1,1))
            x = row.tolist()[:4]
            output_layer, output = self.feed_forward(x)
            self.back_propagation(output_layer, y, 0.01)
        return 0

    def predict(self, X, y):
        print(f'layers: {self.layers}')
        print()
        print(f'weights: {self.weights}')
        percentages, prediction = self.feed_forward(X)
#         print(f'percent: {percentages}')
#         print()
#         print(f'preiction: {prediction}')
#         print(f'y: {y}')
        return prediction

In [4]:
sample = X_train.iloc[0]

In [5]:
new_class = Network(sample ,2, [4,3])

In [6]:
new_class.train(X_train, y_train)

0

In [7]:
X_test.iloc[0]
# ytest.get_dummies

sepal length (cm)    6.2
sepal width (cm)     2.8
petal length (cm)    4.8
petal width (cm)     1.8
Name: 0, dtype: float64

In [8]:
new_class.predict(list(X_test.iloc[11]), np.reshape(list(pd.get_dummies(y_test).iloc[11]), [-1,1]))

layers: [array([[7.2],
       [3.2],
       [6. ],
       [1.8]]), array([[0.99999635],
       [0.9966586 ],
       [0.99996744],
       [0.99992687]]), array([[0.5001598],
       [0.5001598],
       [0.5001598]]), array([[0.50433751],
       [0.49566182],
       [0.49566249]])]

weights: [array([[0.45277356, 0.78712746, 0.89515849, 0.762176  ],
       [0.07904413, 0.75330535, 0.29570398, 0.5245001 ],
       [0.92871655, 0.55000584, 0.24963186, 0.21550217],
       [0.7299736 , 0.2055677 , 0.35709719, 0.81496711]]), array([[0.00042109, 0.00018715, 0.00035091, 0.00010527],
       [0.00042109, 0.00018715, 0.00035091, 0.00010527],
       [0.00042109, 0.00018715, 0.00035091, 0.00010527]]), array([[-0.00504499, -0.00504499, -0.00504499],
       [-0.0049582 , -0.0049582 , -0.0049582 ],
       [ 0.00504499,  0.00504499,  0.00504499]])]


array([0.5044145])